In [1]:
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from torchvision.transforms.functional import crop
import torch
import torchvision.transforms as transforms
import gc
from torch.utils.data import DataLoader,Dataset


In [3]:
def to_float32_and_scale(tensor,global_min,global_max):
    tensor = tensor.float()
    tensor = (tensor - global_min) / (global_max - global_min)
    return tensor

def crop_array(array):
    
    return crop(array, 20, 28, 85, 36)

path_data = os.path.join(os.path.dirname(os.path.dirname((os.getcwd()))),'DadesUAB/Data')

print(path_data)
    
transform = {
            'input': transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(crop_array),
                transforms.Resize((192, 84))]),
            'output': transforms.Compose([
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(1),
                transforms.Lambda(lambda x: to_float32_and_scale(x,0,905)),
                transforms.Resize((192, 84))])
}

max_ir = -np.inf
min_ir = np.inf
max_pm = -np.inf
min_pm = np.inf
images_tensor = {}
images_tensor['input']=[]
images_tensor['output']=[]
folders = os.listdir(path_data)
folders_sorted = sorted(folders)
print(folders_sorted)
for folder in folders_sorted:
    directory = os.path.join(path_data,folder)
    #print(directory)
    pattern = os.path.join(directory, '*IR.png')
    files_ir = glob(pattern)
    files_ir = sorted(files_ir)
    #print(len(files_ir))
    #print(files_ir)
    pattern = os.path.join(directory, '*Pressio.csv')
    files_pm = glob(pattern)
    files_pm = sorted(files_pm)
    #print(len(files_pm))
    #print(files_pm)

    for ir,pm in zip(files_ir,files_pm):
        #print(ir)
        #print(pm)
        ir_array = mpimg.imread(ir)
        array = np.rot90(ir_array, k=1, axes=(1,0))
        array_2 = np.copy(array)
        tensor_final = transform['input'](array_2)
        images_tensor['input'].append(tensor_final)
        ir_array_final = tensor_final.squeeze().numpy()
        pm = pd.read_csv(pm)
        pm_array = pm.to_numpy()
        pm_array = np.rot90(pm_array, k=1, axes=(1,0))
        pm_array_2 = np.copy(pm_array)
        pm_tensor = transform['output'](pm_array_2)
        images_tensor['output'].append(pm_tensor)
        final_array = pm_tensor.squeeze().numpy()
        
        pmin, pmax = final_array.min(), final_array.max()
        min_pm = min(min_pm, pmin)
        max_pm = max(max_pm, pmax)

/home/ggausachs/DadesUAB/Data
['231218151009', '231218171115', '231218184644', '231219111259', '231219121247', '231219125929', '231219160955', '231219163825', '231221152928', '231221160009', '231221173600', '231222090337', '240112122326', '240117120951', '240130104100', '240130110959', '240201120017', '240207155139', '240212155248', '240301122444', '240403123945']


In [23]:
class CustomDataset(Dataset):
    def __init__(self, data_dict):
        self.inputs = data_dict['input']
        self.outputs = data_dict['output']
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        input_tensor = self.inputs[idx]
        output_tensor = self.outputs[idx]
        return input_tensor,output_tensor

In [27]:
from Models import (
    UNet,
    UNet_phy
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAME = "UNet"
name_model = "Model Name"

models = {"UNet": UNet.UNET, "UNet_phy": UNet_phy.UNET_phy}

features = [64,128,256,512]

model = models[MODEL_NAME](1, 1,features).to(DEVICE)

# Load the model
model_file = os.path.join(os.path.join((os.getcwd()),'Models/SavedModels'),name_model)

try:
    model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
    model.to(DEVICE)
except Exception as e:
    raise RuntimeError(f"An error occurred while reading the file: {e}")
else:
    print('Model Loaded')

print(model)

model.eval()
torch.cuda.empty_cache()  # Clean CUDA Cache if used GPU
gc.collect()  # Collect trash to free memory not used


custom_dataset = CustomDataset(images_tensor)

test_loader = DataLoader(custom_dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=True)

total_metric = [0, 0, 0, 0]

with torch.no_grad():
        best_input_img = None
        best_target_img = None
        best_output_img = None
        best_mse = np.inf

        for batch_idx, (input_images, target_images) in enumerate(test_loader, 1):
    
                input_img = input_images.to(DEVICE)
                target_img = target_images.to(DEVICE)
    
                output_img = model(input_img)
    
                for i, metric in enumerate(metrics):
    
                    test_metric = metric(output_img, target_img)
    
                    total_metric[i] += test_metric

                    if i == 0:
                        if best_mse > test_metric:
                            best_input_img = input_img
                            best_target_img = target_img
                            best_output_img = output_img
                            best_mse = test_metric
    
                # Free memory in each iteration
                torch.cuda.empty_cache()  # Clean CUDA Cache if used GPU
                gc.collect()  # Collect trash to free memory not used
            
print('Evaluation completed')
epoch_metric = [(total_metric[0] /
                    len(test_loader)).item(), total_metric[1] / len(test_loader),
                    total_metric[2] / len(test_loader),total_metric[3] / len(test_loader)]
    
print(metrics)
print(epoch_metric)
    
m_str = [f"MSE - {epoch_metric[0]}", f"PerCS - {epoch_metric[1]}", f"MSEeff - {epoch_metric[2]}", f"SSIM - {epoch_metric[3]}"]
    
m_str = ", ".join(m_str)
    
print(m_str)

input_images = best_input_img.squeeze().cpu().numpy()
target_images = best_target_img.squeeze().cpu().numpy()
output_img = best_output_img.squeeze().cpu().numpy()

plt.imshow(input_images)
plt.axis('off')
#plt.savefig(os.path.join("Input_Image.png"))
plt.close()

# Save the target image
plt.imshow(target_images)
plt.axis('off')
#plt.savefig(os.path.join("Target_Image.png"))
plt.close()

# Save the output image
plt.imshow(output_img)
plt.axis('off')
#plt.savefig(os.path.join("Output_Image.png"))
plt.close()

Model Loaded
UNET(
  (ups): ModuleList(
    (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (2): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), str